In [7]:
import pandas as pd

dados = pd.read_csv(r"D:\reconheceprodutos\produtos_vpj.csv",sep=";",encoding='latin-1')

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report


dados = pd.read_csv(r"D:\reconheceprodutos\produtos_vpj.csv", sep=";", encoding='latin-1')


dados['item_descricao'] = dados['item_descricao'].str.lower()


def aplicar_regras_manuais(descricao):
    descricao = descricao.lower()
    if "bacon" in descricao or "linguiça" in descricao:
        grupo = "SUÍNOS - DUROC"
        subgrupo = "LINGUIÇAS"
    elif "cordeiro" in descricao:
        grupo = "OVINOS"
        subgrupo = "COSTELA"
    else:
        grupo, subgrupo = None, None
    return grupo, subgrupo


dados['grupo_regra'], dados['subgrupo_regra'] = zip(*dados['item_descricao'].apply(aplicar_regras_manuais))


X = dados['item_descricao']
y_grupo = dados['descricao_grupo_custo'].where(dados['grupo_regra'].isnull(), dados['grupo_regra'])
y_subgrupo = dados['descricao_subgrupo_custo'].where(dados['subgrupo_regra'].isnull(), dados['subgrupo_regra'])
y = pd.DataFrame({'descricao_grupo_custo': y_grupo, 'descricao_subgrupo_custo': y_subgrupo})


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


y_pred_grupo = [pred[0] for pred in y_pred]
y_pred_subgrupo = [pred[1] for pred in y_pred]


print("Relatório de classificação para descricao_grupo_custo:")
print(classification_report(y_test['descricao_grupo_custo'], y_pred_grupo))

print("\nRelatório de classificação para descricao_subgrupo_custo:")
print(classification_report(y_test['descricao_subgrupo_custo'], y_pred_subgrupo))


def predizer_grupo_subgrupo(descricao_item):
    descricao_item = descricao_item.lower()
    grupo_regra, subgrupo_regra = aplicar_regras_manuais(descricao_item)
    
    
    if grupo_regra and subgrupo_regra:
        return [[grupo_regra, subgrupo_regra]]
    
    return pipeline.predict([descricao_item])


descricao_exemplo = "LINGUICA BACON 500G - CX"
print(predizer_grupo_subgrupo(descricao_exemplo))


Relatório de classificação para descricao_grupo_custo:
                              precision    recall  f1-score   support

             BOVINOS - ANGUS       0.94      0.96      0.95       106
              BOVINOS - ZEBU       0.89      1.00      0.94        24
INDUSTRIALIZADOS - BOV ANGUS       0.00      0.00      0.00         5
 INDUSTRIALIZADOS - BOV ZEBU       0.00      0.00      0.00         1
    INDUSTRIALIZADOS - SUÍNO       1.00      0.11      0.20         9
                      OVINOS       1.00      1.00      1.00        27
                     REVENDA       1.00      1.00      1.00        21
              SUÍNOS - DUROC       0.80      0.97      0.88        33
             SUÍNOS - LEITOA       1.00      1.00      1.00         1
            SUÍNOS - MARROTE       1.00      1.00      1.00         2

                    accuracy                           0.92       229
                   macro avg       0.76      0.70      0.70       229
                weighted avg     

d:\reconheceprodutos\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\reconheceprodutos\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\reconheceprodutos\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\reconheceprodutos\venv\lib\site-pa

In [18]:
print(predizer_grupo_subgrupo(descricao_exemplo))

[['SUÍNOS - DUROC', 'LINGUIÇAS']]


In [68]:
import pandas as pd
import numpy as np
import torch
import joblib
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder


model_grupo_path = 'classifier_grupo_final.pkl'
model_subgrupo_path = 'classifier_subgrupo_final.pkl'
le_grupo_path = 'le_grupo.pkl'
le_subgrupo_path = 'le_subgrupo.pkl'
le_marca_path = 'le_marca.pkl'
tfidf_vectorizer_path = 'tfidf_vectorizer.pkl'
onehot_encoder_path = 'onehot_encoder.pkl'


tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')


def extrair_marca(texto):
    texto = texto.lower()
    if '-' in texto:
        return texto.split('-')[-1].strip()
    else:
        return 'desconhecido'


def extrair_palavras_chave(texto):
    palavras_irrelevantes = set(['cx', 'un', 'g', 'kg', 'ml', 'l', 'mg'])
    palavras = texto.lower().split()
    palavras_filtradas = [palavra for palavra in palavras if palavra not in palavras_irrelevantes and not palavra.isdigit()]
    return ' '.join(palavras_filtradas)


def gerar_embeddings(textos):
    embeddings = []
    batch_size = 16 
    for i in range(0, len(textos), batch_size):
        batch_textos = textos.iloc[i:i+batch_size].tolist()
        inputs = tokenizer(batch_textos, return_tensors='pt', padding=True, truncation=True, max_length=50)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

if (os.path.exists(model_grupo_path) and os.path.exists(model_subgrupo_path) and
    os.path.exists(le_grupo_path) and os.path.exists(le_subgrupo_path) and
    os.path.exists(le_marca_path) and os.path.exists(tfidf_vectorizer_path) and
    os.path.exists(onehot_encoder_path)):
    
    classifier_grupo_final = joblib.load(model_grupo_path)
    classifier_subgrupo_final = joblib.load(model_subgrupo_path)
    le_grupo = joblib.load(le_grupo_path)
    le_subgrupo = joblib.load(le_subgrupo_path)
    le_marca = joblib.load(le_marca_path)
    tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
    onehot_encoder = joblib.load(onehot_encoder_path)
    print("Modelos e encoders carregados dos arquivos .pkl.")
else:
    
    dados = pd.read_csv(r"D:\reconheceprodutos\produtos_vpj.csv", sep=";", encoding='latin-1')

    
    dados['item_descricao'] = dados['item_descricao'].str.lower().str.strip()
    dados['marca'] = dados['item_descricao'].apply(extrair_marca)
    dados['palavras_chave'] = dados['item_descricao'].apply(extrair_palavras_chave)

    
    X_texto = dados['palavras_chave']
    X_marca = dados['marca']
    y = dados[['descricao_grupo_custo', 'descricao_subgrupo_custo']]

    
    le_grupo = LabelEncoder()
    le_subgrupo = LabelEncoder()
    le_marca = LabelEncoder()

    y['descricao_grupo_custo'] = le_grupo.fit_transform(y['descricao_grupo_custo'])
    y['descricao_subgrupo_custo'] = le_subgrupo.fit_transform(y['descricao_subgrupo_custo'])
    X_marca_encoded = le_marca.fit_transform(X_marca)

    
    y_combined = y['descricao_grupo_custo'].astype(str) + "_" + y['descricao_subgrupo_custo'].astype(str)

    
    print("Gerando embeddings, isso pode levar algum tempo...")
    X_embeddings = gerar_embeddings(X_texto)

   
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)
    X_tfidf = tfidf_vectorizer.fit_transform(X_texto).toarray()

    
    onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_marca_onehot = onehot_encoder.fit_transform(X_marca.values.reshape(-1, 1))

    
    X_final = np.hstack((X_embeddings, X_tfidf, X_marca_onehot))

    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    reportes_grupo = []
    reportes_subgrupo = []

   
    for fold, (train_index, test_index) in enumerate(skf.split(X_final, y_combined)):
        print(f"\nIniciando fold {fold + 1}...")
        
        X_train, X_test = X_final[train_index], X_final[test_index]
        y_train, y_test = y.iloc[train_index].copy(), y.iloc[test_index].copy()

        
        classifier_grupo = RandomForestClassifier(random_state=42, n_estimators=500, max_depth=None)
        classifier_grupo.fit(X_train, y_train['descricao_grupo_custo'])

        
        classifier_subgrupo = RandomForestClassifier(random_state=42, n_estimators=500, max_depth=None)
        classifier_subgrupo.fit(X_train, y_train['descricao_subgrupo_custo'])

        
        y_pred_grupo = classifier_grupo.predict(X_test)
        y_pred_subgrupo = classifier_subgrupo.predict(X_test)

        
        y_test['descricao_grupo_custo'] = le_grupo.inverse_transform(y_test['descricao_grupo_custo'])
        y_test['descricao_subgrupo_custo'] = le_subgrupo.inverse_transform(y_test['descricao_subgrupo_custo'])
        y_pred_grupo_decoded = le_grupo.inverse_transform(y_pred_grupo)
        y_pred_subgrupo_decoded = le_subgrupo.inverse_transform(y_pred_subgrupo)

        
        print("\nRelatório de classificação para descricao_grupo_custo nesta divisão:")
        report_grupo = classification_report(y_test['descricao_grupo_custo'], y_pred_grupo_decoded, output_dict=True)
        reportes_grupo.append(report_grupo)
        print(classification_report(y_test['descricao_grupo_custo'], y_pred_grupo_decoded))

        print("\nRelatório de classificação para descricao_subgrupo_custo nesta divisão:")
        report_subgrupo = classification_report(y_test['descricao_subgrupo_custo'], y_pred_subgrupo_decoded, output_dict=True)
        reportes_subgrupo.append(report_subgrupo)
        print(classification_report(y_test['descricao_subgrupo_custo'], y_pred_subgrupo_decoded))

    
    def calcular_media_relatorios(reports):
        
        metrics = {}
        num_reports = len(reports)
        for report in reports:
            for label in report:
                if label not in ['accuracy', 'macro avg', 'weighted avg']:
                    if label not in metrics:
                        metrics[label] = {'precision': report[label]['precision'],
                                          'recall': report[label]['recall'],
                                          'f1-score': report[label]['f1-score'],
                                          'support': report[label]['support']}
                    else:
                        metrics[label]['precision'] += report[label]['precision']
                        metrics[label]['recall'] += report[label]['recall']
                        metrics[label]['f1-score'] += report[label]['f1-score']
                        metrics[label]['support'] += report[label]['support']
        
        for label in metrics:
            metrics[label]['precision'] /= num_reports
            metrics[label]['recall'] /= num_reports
            metrics[label]['f1-score'] /= num_reports
        return metrics

   
    print("\nMédia das métricas para descricao_grupo_custo após K-Fold:")
    media_grupo = calcular_media_relatorios(reportes_grupo)
    print(media_grupo)

    print("\nMédia das métricas para descricao_subgrupo_custo após K-Fold:")
    media_subgrupo = calcular_media_relatorios(reportes_subgrupo)
    print(media_subgrupo)

    
    print("\nTreinando o modelo final em todo o conjunto de dados...")
    
    classifier_grupo_final = RandomForestClassifier(random_state=42, n_estimators=500, max_depth=None)
    classifier_grupo_final.fit(X_final, y['descricao_grupo_custo'])

   
    classifier_subgrupo_final = RandomForestClassifier(random_state=42, n_estimators=500, max_depth=None)
    classifier_subgrupo_final.fit(X_final, y['descricao_subgrupo_custo'])

   
    joblib.dump(classifier_grupo_final, model_grupo_path)
    joblib.dump(classifier_subgrupo_final, model_subgrupo_path)
    joblib.dump(le_grupo, le_grupo_path)
    joblib.dump(le_subgrupo, le_subgrupo_path)
    joblib.dump(le_marca, le_marca_path)
    joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
    joblib.dump(onehot_encoder, onehot_encoder_path)
    print("Modelos e encoders salvos em arquivos .pkl.")


def predizer_grupo_subgrupo(descricao_item):
    descricao_item_original = descricao_item 
    descricao_item = descricao_item.lower().strip()
    marca = extrair_marca(descricao_item)
    palavras_chave = extrair_palavras_chave(descricao_item)

    
    inputs = tokenizer([palavras_chave], return_tensors='pt', padding=True, truncation=True, max_length=50)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].numpy()

    
    tfidf_vector = tfidf_vectorizer.transform([palavras_chave]).toarray()

    
    if marca in onehot_encoder.categories_[0]:
        marca_encoded = onehot_encoder.transform([[marca]])
    else:
        marca_encoded = np.zeros((1, len(onehot_encoder.categories_[0])))

  
    embedding_final = np.hstack((embedding, tfidf_vector, marca_encoded))

  
    pred_grupo = classifier_grupo_final.predict(embedding_final)
    pred_subgrupo = classifier_subgrupo_final.predict(embedding_final)

    
    proba_subgrupo = classifier_subgrupo_final.predict_proba(embedding_final)
    max_proba_subgrupo = np.max(proba_subgrupo)

   
    threshold = 0.2 

    
    grupo = le_grupo.inverse_transform(pred_grupo)[0]
    if max_proba_subgrupo >= threshold:
        subgrupo = le_subgrupo.inverse_transform(pred_subgrupo)[0]
    else:
        subgrupo = 'CATEGORIA NAO EXISTENTE'
    return [[descricao_item_original, grupo, subgrupo]]


descricao_exemplo = "DOCE DE COCO CORTE 700G CX 15 UN - SWEET VALLEY"
print("\nPredição para a descrição do item:")
resultado = predizer_grupo_subgrupo(descricao_exemplo)
print(resultado)



Modelos e encoders carregados dos arquivos .pkl.

Predição para a descrição do item:
[['DOCE DE COCO CORTE 700G CX 15 UN - SWEET VALLEY', 'REVENDA', 'REVENDA - SWEET VALLEY']]


In [70]:
descricao_exemplo = "CARNE MOIDA CONGELADA CORDEIRO 700 G"
print(predizer_grupo_subgrupo(descricao_exemplo))

[['CARNE MOIDA CONGELADA CORDEIRO 700 G', 'OVINOS', 'CATEGORIA NAO EXISTENTE']]


In [ ]:
MIUDOS - FIGADO DE CORDEIRO CONG.
CARNE MOIDA CONGELADA CORDEIRO 700 G
DOCE DE COCO CORTE 700G CX 15 UN - SWEET VALLEY